In [3]:
# import preprocess as prep
import SLR_model
import numpy as np
from tensorflow import keras
from datetime import datetime
import os
import importlib
from matplotlib import pyplot as plt

# preprocess / load preprocessed data (preprocess.py)
# mini batching
import preprocess
# load model(can quickly change model by changing imported model)
# train / log
# generate checkpoints

# this will output 15 results/sec, how are we gonna handle it?

In [4]:
 # reload model file
importlib.reload(SLR_model)
importlib.reload(preprocess)

<module 'preprocess' from 'C:\\Users\\shd\\Documents\\capstone\\BomNae-SLR\\preprocess.py'>

In [29]:
# utility functions

def bin2num(bin_arr):
    # big endian
    # lower index > higher exponent
    num = 0
    for val in bin_arr:
        num = (num << 1) | val
    return num

# def num2bin(num, arr_len = -1):
#     # big endian
#     # lower index > higher exponent
#     bin_arr = [int(x) for x in bin(num)[2:]]
#     diff = arr_len - len(bin_arr)
#     if diff < 1:
#         return bin_arr
#     else:
#         pad = [0] * diff
#         return pad+bin_arr
    
def num_arr2bin(num_arr, out_len = -1):
    # big endian
    # lower index > higher exponent
    res = []
    for num in num_arr:
        str_list = list(np.binary_repr(num).zfill(out_len))
        bin_arr = np.array(str_list, dtype=np.float32)
        res.append(bin_arr)
    return np.array(res)

def serialize(vids, stride = 1):
    """input shape: (load_size, frames)\n
    ouput shape: (load_size, input_seq_size, 63 or 32, frames)"""
    each_size = []
    x_res = []
    for vid in vids:
        window_len = 0
        start = 0
        while (start + 63) < len(vid):
            x_res.append(vid[start: start+63: stride])
            window_len += 1
            start += 6
        each_size.append(window_len)
    return np.array(x_res), each_size

In [30]:
# Configurations
load_size = 256 # number of data to be loaded at once
epochs = 8
batch_size = 16
save_dir = "checkpoints"
load_path = "checkpoints"


In [31]:
# checkpoint_filepath = '/tmp/ckpt/checkpoint.model.keras'
model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=f'{save_dir}.keras',
    monitor='binary_accuracy',
    mode='max',
    save_freq='epoch',
    save_best_only=True)

In [32]:
l_raw, r_raw, p_raw, y_raw ,check_raw= preprocess.load_word(1,1,load_size)

In [33]:
print(l_raw[0].shape)

(138, 4, 5, 3)


In [34]:
# dummy x_raw data
l_raw = []
for i in range(load_size):
    h_elem = np.zeros(shape=(4,5,3))
    # p_elem = np.zeros(shape=(3,10))
    # comb_elem = [h_elem, h_elem, p_elem]
    x_elem = []
    for j in range(np.random.randint(70,71)): # frame size
        x_elem.append(h_elem)
        # x_elem.append(comb_elem)
    l_raw.append(x_elem)
r_raw = []
for i in range(load_size):
    h_elem = np.zeros(shape=(4,5,3))
    # p_elem = np.zeros(shape=(3,10))
    # comb_elem = [h_elem, h_elem, p_elem]
    x_elem = []
    for j in range(np.random.randint(70,71)): # frame size
        x_elem.append(h_elem)
        # x_elem.append(comb_elem)
    r_raw.append(x_elem)
p_raw = []
for i in range(load_size):
    # h_elem = np.zeros(shape=(4,5,3))
    p_elem = np.zeros(shape=(3,10))
    # comb_elem = [h_elem, h_elem, p_elem]
    x_elem = []
    for j in range(np.random.randint(70,71)): # frame size
        # x_elem.append(h_elem)
        x_elem.append(p_elem)
        # x_elem.append(comb_elem)
    p_raw.append(x_elem)

# dummy y_raw data
y_raw = np.random.randint(0,1024,load_size)

In [35]:
l_train, each = serialize(l_raw)
r_train, each = serialize(r_raw)
p_train, each = serialize(p_raw, stride=2)
y_train = np.repeat(y_raw, each)
y_train = num_arr2bin(y_train, 12)

In [36]:
# dummy data for model testing
# hand input shape: (batch, time, h, w, channels)
# pose input shape: (batch, time, channel, features)
# hand_l_dummy = np.zeros(shape=(load_size, 63, 4, 5, 3))
# hand_r_dummy = np.zeros(shape=(load_size, 63, 4, 5, 3))
# pose_dummy = np.zeros(shape=(load_size, 32, 3, 10))
# x_dummy = (hand_l_dummy, hand_r_dummy, pose_dummy)
# y_dummy1 = np.ones(shape=(load_size, 12))
# y_dummy0 = np.zeros(shape=(load_size, 12))

In [37]:
x_train = (l_train, r_train, p_train)


In [38]:
x_train[0].shape

(512, 63, 4, 5, 3)

In [39]:
 y_train.shape

(512, 12)

In [40]:
model = SLR_model.get_model()

In [41]:
model = SLR_model.load_model(load_path)

ValueError: File format not supported: filepath=checkpoints. Keras 3 only supports V3 `.keras` files and legacy H5 format files (`.h5` extension). Note that the legacy SavedModel format is not supported by `load_model()` in Keras 3. In order to reload a TensorFlow SavedModel as an inference-only layer in Keras 3, use `keras.layers.TFSMLayer(checkpoints, call_endpoint='serving_default')` (note that your `call_endpoint` might have a different name).

In [42]:
hist = model.fit(x_train, y_train, epochs=epochs, batch_size= batch_size, callbacks=[model_checkpoint_callback])

Epoch 1/8
 1/32 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step

ValueError: Cannot get result() since the metric has not yet been built.

In [14]:
plt.plot(range(len(hist.history['loss'])), hist.history['loss'])
plt.scatter(range(len(hist.history['loss'])), hist.history['loss'])
plt.xlabel("epochs")
plt.ylabel("loss")
plt.legend(["loss"])
plt.ylim((0,1))
plt.show()

NameError: name 'hist' is not defined

In [84]:
importlib.reload(SLR_model)
model = SLR_model.get_model()

In [85]:
# pred = model.predict(x_dummy, batch_size=1)

In [86]:
eval_res = model.evaluate(x_train, y_train, batch_size= batch_size)

32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 935us/step - binary_accuracy: 0.5969 - loss: 0.6931


In [87]:
print(eval_res)

[0.6931474804878235, 0.5989583134651184]


In [88]:
model.metrics_names

['loss', 'compile_metrics']

In [89]:
pred[0]

NameError: name 'pred' is not defined

In [ ]:
save_name = "asdf_test.keras"
save_path = os.path.join(save_dir, save_name)

In [ ]:
model.save_model(save_path)

In [ ]:
model.load_model(save_path)